In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
from intent_classification_helper import *

In [36]:
np.ones((2,2))

array([[1., 1.],
       [1., 1.]])

In [42]:
f = open('/content/drive/MyDrive/nlp_datasets/CLINC150/clinc150_uci/data_full.json')
data = json.load(f)

train_df = pd.DataFrame.from_dict(data['train'])
train_df.columns = ['message', 'intent']

val_df = pd.DataFrame.from_dict(data['val'])
val_df.columns = ['message', 'intent']

test_df = pd.DataFrame.from_dict(data['test'])
test_df.columns = ['message', 'intent']

print('number of intent classes in training set: ', len(set(train_df['intent'])))
print('number of intent classes in val set: ', len(set(val_df['intent'])))
print('number of intent classes in test set: ', len(set(test_df['intent'])))

number of intent classes in training set:  150
number of intent classes in val set:  150
number of intent classes in test set:  150


In [43]:
# encode label
le = LabelEncoder()
train_df['intent'] = le.fit_transform(train_df['intent'])
val_df['intent'] = le.fit_transform(val_df['intent'])
test_df['intent'] = le.fit_transform(test_df['intent'])

onehot_encoder = OneHotEncoder()
y_train = onehot_encoder.fit_transform(train_df['intent'].values.reshape(-1, 1)).todense()
y_val = onehot_encoder.fit_transform(val_df['intent'].values.reshape(-1, 1)).todense()
y_test = onehot_encoder.fit_transform(test_df['intent'].values.reshape(-1, 1)).todense()

# preprocess text
print('Preprocessing text on training set...')
preprocess_text(train_df, 'message')

print('Preprocessing text on val set...')
preprocess_text(val_df, 'message')

print('Preprocessing text on test set...')
preprocess_text(test_df, 'message')

x_train = train_df['message'].to_list()
x_val = val_df['message'].to_list()
x_test = test_df['message'].to_list()

Preprocessing text on training set...


Start text preprocessing: 
--------------------------
Converting to lowercase...
--------------------------
Removing html tags...
--------------------------
Removing nonword characters...
--------------------------
Removing stopwords...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Text preprocessing completed.


Preprocessing text on val set...


Start text preprocessing: 
--------------------------
Converting to lowercase...
--------------------------
Removing html tags...
--------------------------
Removing nonword characters...
--------------------------
Removing stopwords...
Text preprocessing completed.


Preprocessing text on test set...


Start text preprocessing: 
--------------------------
Converting to lowercase...
--------------------------
Removing html tags...
--------------------------
Removing nonword characters...
--------------------------
Removing stopwords...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Text preprocessing completed.




In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

# try different number of features
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=40)
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=100)
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=200)
#tv = TfidfVectorizer(max_df=1.0, min_df=0, max_features=1000)
tv = TfidfVectorizer(max_df=1.0, min_df=0)

tv.fit(x_train)
x_train = tv.transform(x_train).toarray()
x_val = tv.transform(x_val).toarray()
x_test = tv.transform(x_test).toarray()

vocab = tv.get_feature_names_out()

print(pd.DataFrame(x_train, columns=vocab))
print('TF-IDF vocabulary size: ', len(vocab))

        00  000  005  00am  00pm   01   02   03   05  098098  ...  zesty  \
0      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
1      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
2      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
3      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
4      0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
...    ...  ...  ...   ...   ...  ...  ...  ...  ...     ...  ...    ...   
14995  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
14996  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
14997  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
14998  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   
14999  0.0  0.0  0.0   0.0   0.0  0.0  0.0  0.0  0.0     0.0  ...    0.0   

       zeus  zion  zippy  zippys  ziti  zombie  zone  zoo  zulu  
0       0.0   0.0    

In [18]:
cnn(x_train_cnn=x_train, y_train_cnn=y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val), feature_numbers=len(vocab))

Epoch 1/2
469/469 [==============================] - 37s 78ms/step - loss: 5.0025 - accuracy: 0.0077 - val_loss: 4.9598 - val_accuracy: 0.0083
Epoch 2/2
469/469 [==============================] - 34s 73ms/step - loss: 4.8665 - accuracy: 0.0146 - val_loss: 4.8525 - val_accuracy: 0.0093


In [29]:
cnn(x_train_cnn=x_train, y_train_cnn=y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val), feature_numbers=len(vocab))

Epoch 1/20
469/469 [==============================] - 33s 70ms/step - loss: 5.0009 - accuracy: 0.0065 - val_loss: 4.9486 - val_accuracy: 0.0090
Epoch 2/20
469/469 [==============================] - 35s 74ms/step - loss: 4.8518 - accuracy: 0.0129 - val_loss: 4.8493 - val_accuracy: 0.0133
Epoch 3/20
469/469 [==============================] - 35s 75ms/step - loss: 4.8094 - accuracy: 0.0147 - val_loss: 4.8544 - val_accuracy: 0.0123
Epoch 4/20
469/469 [==============================] - 37s 79ms/step - loss: 4.8050 - accuracy: 0.0141 - val_loss: 4.8427 - val_accuracy: 0.0127
Epoch 5/20
469/469 [==============================] - 35s 75ms/step - loss: 4.7985 - accuracy: 0.0156 - val_loss: 4.8488 - val_accuracy: 0.0163
Epoch 6/20
469/469 [==============================] - 35s 75ms/step - loss: 4.7927 - accuracy: 0.0157 - val_loss: 4.8320 - val_accuracy: 0.0153
Epoch 7/20
469/469 [==============================] - 35s 75ms/step - loss: 4.7872 - accuracy: 0.0154 - val_loss: 4.8521 - val_accuracy:

KeyboardInterrupt: ignored

In [34]:
cnn(x_train_cnn=x_train, y_train_cnn=y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val), feature_numbers=len(vocab))

Epoch 1/20
469/469 [==============================] - 36s 76ms/step - loss: 5.0031 - accuracy: 0.0057 - val_loss: 4.9625 - val_accuracy: 0.0070
Epoch 2/20
469/469 [==============================] - 37s 79ms/step - loss: 4.8685 - accuracy: 0.0111 - val_loss: 4.8463 - val_accuracy: 0.0137
Epoch 3/20
469/469 [==============================] - 38s 80ms/step - loss: 4.8146 - accuracy: 0.0135 - val_loss: 4.8498 - val_accuracy: 0.0133
Epoch 4/20
469/469 [==============================] - 36s 77ms/step - loss: 4.8078 - accuracy: 0.0144 - val_loss: 4.8402 - val_accuracy: 0.0123
Epoch 5/20
469/469 [==============================] - 36s 78ms/step - loss: 4.8041 - accuracy: 0.0141 - val_loss: 4.8642 - val_accuracy: 0.0160
Epoch 6/20
469/469 [==============================] - 36s 76ms/step - loss: 4.7973 - accuracy: 0.0162 - val_loss: 4.8438 - val_accuracy: 0.0123
Epoch 7/20
469/469 [==============================] - 35s 75ms/step - loss: 4.7923 - accuracy: 0.0153 - val_loss: 4.8312 - val_accuracy:

KeyboardInterrupt: ignored

In [40]:
cnn(x_train_cnn=x_train, y_train_cnn=y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val), feature_numbers=len(vocab))

Epoch 1/20
469/469 [==============================] - 37s 76ms/step - loss: 5.0037 - accuracy: 0.0069 - val_loss: 4.9607 - val_accuracy: 0.0093
Epoch 2/20
469/469 [==============================] - 37s 78ms/step - loss: 4.8623 - accuracy: 0.0121 - val_loss: 4.8449 - val_accuracy: 0.0143
Epoch 3/20
469/469 [==============================] - 35s 74ms/step - loss: 4.8127 - accuracy: 0.0139 - val_loss: 4.8558 - val_accuracy: 0.0133
Epoch 4/20
469/469 [==============================] - 35s 74ms/step - loss: 4.8057 - accuracy: 0.0149 - val_loss: 4.8509 - val_accuracy: 0.0150
Epoch 5/20
469/469 [==============================] - 35s 74ms/step - loss: 4.7990 - accuracy: 0.0149 - val_loss: 4.8558 - val_accuracy: 0.0147
Epoch 6/20
469/469 [==============================] - 35s 74ms/step - loss: 4.7925 - accuracy: 0.0153 - val_loss: 4.8504 - val_accuracy: 0.0147
Epoch 7/20
469/469 [==============================] - 34s 73ms/step - loss: 4.7922 - accuracy: 0.0169 - val_loss: 4.8567 - val_accuracy:

KeyboardInterrupt: ignored

In [45]:
cnn(x_train_cnn=x_train, y_train_cnn=y_train, batch_size=32, epochs=20, validation_data=(x_val, y_val), feature_numbers=len(vocab))

Epoch 1/20
469/469 [==============================] - 41s 86ms/step - loss: 5.0016 - accuracy: 0.0069 - val_loss: 4.9547 - val_accuracy: 0.0097
Epoch 2/20
469/469 [==============================] - 33s 71ms/step - loss: 4.8670 - accuracy: 0.0129 - val_loss: 4.8455 - val_accuracy: 0.0123
Epoch 3/20
469/469 [==============================] - 33s 71ms/step - loss: 4.8151 - accuracy: 0.0159 - val_loss: 4.8538 - val_accuracy: 0.0147
Epoch 4/20
469/469 [==============================] - 35s 75ms/step - loss: 4.8065 - accuracy: 0.0153 - val_loss: 4.8422 - val_accuracy: 0.0137
Epoch 5/20
469/469 [==============================] - 35s 74ms/step - loss: 4.8018 - accuracy: 0.0157 - val_loss: 4.8332 - val_accuracy: 0.0173
Epoch 6/20
469/469 [==============================] - 34s 72ms/step - loss: 4.7942 - accuracy: 0.0165 - val_loss: 4.8407 - val_accuracy: 0.0147
Epoch 7/20
469/469 [==============================] - 34s 72ms/step - loss: 4.7880 - accuracy: 0.0163 - val_loss: 4.8479 - val_accuracy:

KeyboardInterrupt: ignored